System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 6 on 6 virtual cores
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 6


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw5`


Status `~/Documents/UCLA_files/course_work/BIS_M257/hw/biostat-257-2023-spring/hw5/Project.toml`
  [1e616198] COSMO v0.8.7
  [9961bab8] Cbc v1.1.1
  [f65535da] Convex v0.15.3
  [a93c6f00] DataFrames v1.5.0
  [2e9cd046] Gurobi v1.0.1
  [87dc4568] HiGHS v1.5.1
⌃ [b99e6be6] Hypatia v0.7.2
⌃ [4076af6c] JuMP v1.11.0
  [b8f27783] MathOptInterface v1.16.0
  [1ec41992] MosekTools v0.15.0
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.2.4
  [3eaba693] StatsModels v0.7.2
Info Packages marked with ⌃ have new versions available and may be upgradable.


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to Optimal design

Consider a linear model
\begin{align*}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{align*}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$

\begin{align*}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{align*}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{align*}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{align*}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{align*}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{align*}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company ask you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $C$ with each row a unique design point.

In [3]:
lv_A = ["A1", "A2", "A3"]
lv_B = ["B1", "B2", "B3", "B4"]
ref_A = "A1"; ref_B = "B1"
lv_design = [(a, b) for a in lv_A, b in lv_B if !(a == "A3" && b == "B4")]

row  = zeros(Int, length(lv_A) + length(lv_B) - 2)
nrow = length(lv_design)
C = zeros(Int, nrow, length(row))

for i in 1:nrow
    # reset row
    row .= zeros(Int, length(lv_A) + length(lv_B) - 2)
    
    # dummy coding for A
    if lv_design[i][1] == ref_A 
        row[1:length(lv_A) - 1] .= 0
    else
        row[findfirst(x -> x == lv_design[i][1], lv_A[2:3])] = 1
    end
    
    # dummy coding for B
    if lv_design[i][2] == ref_B
        row[length(lv_A):end] .= 0
    else
        row[findfirst(x -> x == lv_design[i][2], lv_B[2:4]) + length(lv_A) - 1] = 1
    end
    
    C[i, :] = row
end

# add intercepts
C = [ones(Int, nrow) C]

11×6 Matrix{Int64}:
 1  0  0  0  0  0
 1  1  0  0  0  0
 1  0  1  0  0  0
 1  0  0  1  0  0
 1  1  0  1  0  0
 1  0  1  1  0  0
 1  0  0  0  1  0
 1  1  0  0  1  0
 1  0  1  0  1  0
 1  0  0  0  0  1
 1  1  0  0  0  1

## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got, which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.272 │   0.200 │       8 │      27 │      20 │
│      A2B1 │   0.082 │   0.152 │   0.101 │       8 │      15 │      10 │
│      A3B1 │   0.097 │   0.114 │   0.104 │      10 │      11 │      10 │
│      A1B2 │   0.082 │   0.057 │   0.086 │       8 │       6 │       9 │
│      A2B2 │   0.082 │   0.039 │   0.051 │       8 │       4 │       5 │
│      A3B2 │   0.097 │   0.057 │   0.068 │      10 │       6 │       7 │
│      A1B3 │   0.082 │   0.057 │   0.086 │       8 │       6 │       9 │
│      A2B3 │   0.082 │   0.039 │   0.051 │       8 │       4 │       5 │
│      A3B3 │   0.097 │   0.057 │   0.068 │      10 │       6 │       7 │
│      A1B4 │   0.109 │   0.081 │   0.106 │      11 │       8 │      11 │
│      A2B4 │   0.109 │   0.073 │   0.080 │      11 │       7 │       8 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

### D-optimal approximate design

In [4]:
# define the optimization problem
using Convex, LinearAlgebra

p̂ = Variable(length(lv_design))   # proportions of each design to be estimated
# define D-optimal design
problemD = maximize(logdet(C' * diagm(p̂) * C))       # objective function
# problem constraints
problemD.constraints += sum(p̂) == 1 
problemD.constraints += p̂ >= 0      

2-element Vector{Constraint}:
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 869…746)
└─ 1
 >= constraint (affine)
├─ 11-element real variable (id: 869…746)
└─ 0

In [5]:
using COSMO

solver = COSMO.Optimizer()
solve!(problemD, solver)

------------------------------------------------------------------
          COSMO v0.8.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{93},
          constraints: A ∈ R^{262x93} (283 nnz),
          matrix size to factor: 355x355,
          Floating-point precision: Float64
Sets:     ZeroSet of dim: 134
          DensePsdConeTriangle of dim: 78 (12x12)
          PsdConeTriangle of dim: 15 (5x5)
          Nonnegatives of dim: 11
          PsdConeTriangle of dim: 6 (3x3)
          ... and 6 more
Decomp:   Num of original PSD cones: 2
          Num decomposable PSD cones: 1
          Num PSD cones after decomposition: 3
          Merge Strategy: CliqueGraphMerge
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,



Acc:      Anderson Type2{QRDecomp},
          Memory size = 15, RestartedMemory,	
          Safeguarded: true, tol: 2.0
Setup Time: 359.14ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-9.6009e+01	1.4121e+01	5.9990e-01	1.0000e-01


25	 8.5879e-01	1.6876e-01	1.2584e-02	1.0000e-01
50	 3.4329e+00	7.7133e-02	1.8690e-03	1.0000e-01
75	 5.8342e+00	3.9683e-02	3.6902e-03	1.0000e-01
100	 6.6247e+00	2.4936e-02	4.2324e-04	1.0000e-01
125	 6.6701e+00	2.5509e-02	1.2895e-03	1.0000e-01
150	 6.8668e+00	2.1209e-02	7.3481e-04	1.0000e-01
175	 7.7755e+00	1.1744e-02	1.9629e-04	1.0000e-01
200	 7.8205e+00	1.2082e-02	4.4003e-05	1.0000e-01
225	 8.0483e+00	9.1542e-03	4.8153e-04	1.0000e-01
250	 8.0970e+00	9.3261e-03	1.0911e-03	5.9439e-01


275	 8.6568e+00	7.2422e-03	7.8998e-03	5.9439e-01
300	 8.8174e+00	5.4036e-03	1.5872e-03	5.9439e-01
325	 9.0007e+00	2.7150e-03	1.8441e-03	5.9439e-01
350	 9.0698e+00	1.4463e-03	2.1541e-04	5.9439e-01
375	 9.0439e+00	1.0052e-03	2.2881e-03	5.9439e-01
400	 9.0212e+00	6.3932e-04	3.4514e-04	5.9439e-01
425	 9.0006e+00	5.6167e-04	3.3039e-04	5.9439e-01
450	 8.9950e+00	4.3984e-04	1.0361e-04	5.9439e-01
475	 8.9942e+00	3.5602e-04	4.1576e-04	5.9439e-01
500	 8.9946e+00	2.9082e-04	1.0906e-04	5.9439e-01
525	 8.9944e+00	2.1802e-04	5.5301e-04	5.9439e-01
550	 8.9931e+00	1.6503e-04	1.4654e-03	5.9439e-01
575	 8.9893e+00	1.6856e-04	6.4776e-04	5.9439e-01
600	 8.9873e+00	8.8042e-05	1.1744e-03	5.9439e-01
625	 8.9862e+00	5.7349e-05	1.2307e-05	5.9439e-01
650	 8.9850e+00	5.7349e-05	1.6296e-05	5.9439e-01
675	 8.9847e+00	4.8150e-05	2.0397e-04	5.9439e-01
700	 8.9846e+00	3.5048e-05	4.8543e-05	5.9439e-01
725	 8.9849e+00	2.4662e-05	9.4032e-05	5.9439e-01
750	 8.9853e+00	2.8224e-05	1.8968e-04	5.9439e-01
775	 8.9860e+00	2.49


------------------------------------------------------------------
>>> Results
Status: Solved
Iterations: 826 (incl. 26 safeguarding iter)
Optimal objective: 8.987
Runtime: 3.587s (3587.39ms)



In [6]:
@show problemD.status
p̂.value

problemD.status = MathOptInterface.OPTIMAL


11×1 Matrix{Float64}:
 0.08198367711225783
 0.0819519977897672
 0.09679368618602704
 0.08195112396292145
 0.0819656232427522
 0.09682788824260888
 0.08193513776281283
 0.0819376761598751
 0.09680687898486838
 0.1089113098093264
 0.10893491129540801

I got the same answer as Dr. Zhou's suggested results for D-optimal design.

### E-optimal approximate design

In [7]:
# define E-optimal design
problemE = maximize(eigmin(C' * diagm(p̂) * C))       # objective function
# problem constraints
problemE.constraints += sum(p̂) == 1 
problemE.constraints += p̂ >= 0      

2-element Vector{Constraint}:
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 869…746)
└─ 1
 >= constraint (affine)
├─ 11-element real variable (id: 869…746)
└─ 0

In [8]:
solver = COSMO.Optimizer()
solve!(problemE, solver)

------------------------------------------------------------------
          COSMO v0.8.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{16},
          constraints: A ∈ R^{49x16} (82 nnz),
          matrix size to factor: 65x65,
          Floating-point precision: Float64
Sets:     ZeroSet of dim: 17
          PsdConeTriangle of dim: 15 (5x5)
          Nonnegatives of dim: 11
          PsdConeTriangle of dim: 6 (3x3)
Decomp:   Num of original PSD cones: 1
          Num decomposable PSD cones: 1
          Num PSD cones after decomposition: 2
          Merge Strategy: CliqueGraphMerge
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000,
          scaling iter = 10 (on),
          check termination every 25 iter,
   

In [9]:
@show problemE.status
@show eigmax = 1 / problemE.optval
p̂.value

problemE.status = MathOptInterface.OPTIMAL
eigmax = 1 / problemE.optval = 

12.999999990016665


11×1 Matrix{Float64}:
 0.3104287228983093
 0.11303832307063377
 0.1149944925719544
 0.0824744597181178
 0.013874447817003251
 0.057497246293577824
 0.0824744597181007
 0.013874447817011186
 0.05749724629358936
 0.06855672612885096
 0.08528942769691288

I got a different answer for the E-optimal design, but the max eigenvalue is the same so the reason for that is E-optimal gives multiple optimal solutions.

### A-optimal approximate design

In [10]:
# define A-optimal design
p̂ = Variable(length(lv_design))  
r = size(C, 2) # number of columns of C
Y = Variable(r, r) # Shur complement
problemA = minimize(tr(Y)) # objective function
# problem constraints
problemA.constraints += ([C' * diagm(p̂) * C I(r); I(r) Y] ⪰ 0)
problemA.constraints += sum(p̂) == 1 
problemA.constraints += p̂ >= 0 

3-element Vector{Constraint}:
 sdp constraint (affine)
└─ transpose (affine; real)
   └─ hcat (affine; real)
      ├─ transpose (affine; real)
      │  └─ …
      └─ transpose (affine; real)
         └─ …
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 582…706)
└─ 1
 >= constraint (affine)
├─ 11-element real variable (id: 582…706)
└─ 0

In [11]:
solver = COSMO.Optimizer()
solve!(problemA, solver)

------------------------------------------------------------------
          COSMO v0.8.7 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2022
------------------------------------------------------------------

Problem:  x ∈ R^{89},
          constraints: A ∈ R^{176x89} (208 nnz),
          matrix size to factor: 265x265,
          Floating-point precision: Float64
Sets:     ZeroSet of dim: 68
          PsdConeTriangle of dim: 36 (8x8)
          PsdConeTriangle of dim: 21 (6x6)
          PsdConeTriangle of dim: 15 (5x5)
          PsdConeTriangle of dim: 15 (5x5)
          ... and 2 more
Decomp:   Num of original PSD cones: 1
          Num decomposable PSD cones: 1
          Num PSD cones after decomposition: 5
          Merge Strategy: CliqueGraphMerge
Settings: ϵ_abs = 1.0e-05, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-04, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 5000

In [12]:
@show problemA.status
p̂.value

problemA.status = MathOptInterface.OPTIMAL


11×1 Matrix{Float64}:
 0.19990413646111832
 0.10053100947958737
 0.10416514732446445
 0.08637869473209123
 0.05084998157162025
 0.06773931394443254
 0.08637596837979428
 0.05085019442857968
 0.06773908643899029
 0.10556850270663867
 0.07989796020701016

Here I got the same estimated proportions as Dr. Zhou's suggested results for A-optimal design.

## Q3 (30 pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got. Apparently I haven't got the E-optimal design right yet.

```
Exact Optimal Design
┌───────────┬───────┬───────┬───────┐
│ design_pt │ D_opt │ E_opt │ A_opt │
│    String │ Int64 │ Int64 │ Int64 │
├───────────┼───────┼───────┼───────┤
│      A1B1 │     8 │    90 │    20 │
│      A2B1 │     8 │     1 │    10 │
│      A3B1 │    10 │     1 │    10 │
│      A1B2 │     8 │     1 │     9 │
│      A2B2 │     8 │     1 │     5 │
│      A3B2 │    10 │     1 │     7 │
│      A1B3 │     8 │     1 │     9 │
│      A2B3 │     8 │     1 │     5 │
│      A3B3 │    10 │     1 │     7 │
│      A1B4 │    11 │     1 │    10 │
│      A2B4 │    11 │     1 │     8 │
└───────────┴───────┴───────┴───────┘
```

### D-optimal exact design

In [13]:
# define the number of units for each design to be estimated
n̂ = Variable(length(lv_design), IntVar)
# define D-optimal design
problemD_MI = maximize(logdet(C' * diagm(n̂) * C))       # objective function
# problem constraints
problemD_MI.constraints += sum(n̂) == 100
problemD_MI.constraints += n̂ >= 0      

2-element Vector{Constraint}:
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 180…069)
└─ 100
 >= constraint (affine)
├─ 11-element real variable (id: 180…069)
└─ 0

In [17]:
using Pajarito, MathOptInterface, JuMP # use Pajarito to solve mixed-integer problem
using Gurobi, MosekTools # choose two commercial solvers for the best performance
pajarito_solver = Pajarito.Optimizer()
MOI = MathOptInterface
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("oa_solver"), 
        optimizer_with_attributes(Gurobi.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("conic_solver"), 
        optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("verbose"), false)
# MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("iteration_limit"), 100000)

solve!(problemD_MI, pajarito_solver)
problemD_MI.status

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-24


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: integer variable solution is not integral to tolerance tol_feas
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:441
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: integer variable solution is not integral to tolerance tol_feas
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:441
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


OPTIMAL::TerminationStatusCode = 1

In [18]:
n̂.value

11×1 Matrix{Float64}:
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
  9.0
  8.0
  9.0
 11.0
 11.0

I achieved the same results as the suggested answer.

### E-optimal exact design

In [19]:
# define E-optimal design
problemE_MI = maximize(eigmin(C' * diagm(n̂) * C))       # objective function
# problem constraints
problemE_MI.constraints += sum(n̂) == 100
problemE_MI.constraints += n̂ >= 0

2-element Vector{Constraint}:
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 180…069)
└─ 100
 >= constraint (affine)
├─ 11-element real variable (id: 180…069)
└─ 0

In [20]:
pajarito_solver = Pajarito.Optimizer()
MOI = MathOptInterface
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("oa_solver"), 
        optimizer_with_attributes(Gurobi.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("conic_solver"), 
        optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("verbose"), false)

solve!(problemE_MI, pajarito_solver)
problemE_MI.status

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-24
new incumbent


new incumbent


new incumbent


new incumbent


OPTIMAL::TerminationStatusCode = 1

In [21]:
n̂.value

11×1 Matrix{Float64}:
 34.0
 10.0
 10.0
  9.0
  0.0
  6.0
  8.0
  4.0
  4.0
  9.0
  6.0

Here's the answer I achieved for the E-optimal exact design, it looks different from the approximate design, but the numbers look reasonable.

### A-optimal exact design

In [22]:
# define A-optimal design
r = size(C, 2) # number of columns of C
Y = Variable(r, r) # Shur complement
problemA_MI = minimize(tr(Y)) # objective function
# problem constraints
problemA_MI.constraints += ([C' * diagm(n̂) * C I(r); I(r) Y] ⪰ 0)
problemA_MI.constraints += sum(n̂) == 100 
problemA_MI.constraints += n̂ >= 0 

3-element Vector{Constraint}:
 sdp constraint (affine)
└─ transpose (affine; real)
   └─ hcat (affine; real)
      ├─ transpose (affine; real)
      │  └─ …
      └─ transpose (affine; real)
         └─ …
 == constraint (affine)
├─ sum (affine; real)
│  └─ 11-element real variable (id: 180…069)
└─ 100
 >= constraint (affine)
├─ 11-element real variable (id: 180…069)
└─ 0

In [23]:
pajarito_solver = Pajarito.Optimizer()
MOI = MathOptInterface
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("oa_solver"), 
        optimizer_with_attributes(Gurobi.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("conic_solver"), 
        optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true))
MOI.set(pajarito_solver, MOI.RawOptimizerAttribute("verbose"), false)

solve!(problemA_MI, pajarito_solver)
problemA_MI.status

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-24
norm of dual is 1.4714835850808552e16


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito /Users/yuyuanlin/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


OPTIMAL::TerminationStatusCode = 1

In [24]:
n̂.value

11×1 Matrix{Float64}:
 20.0
 10.0
 10.0
  9.0
  5.0
  7.0
  8.0
  5.0
  7.0
 11.0
  8.0

I achieved the same answer as the suggested results for A-optimal exact design.

## Q4 (30 bonus points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.